In [2]:
%lsmagic
%reload_ext sql
%sql postgresql://postgres@localhost:5432/

# Analysis

## 1. Solution Plan

### 1.1. Category insights

1. Create a base dataset and join all relevant tables
`complete_joint_dataset`

2. Calculate customer rental counts for each category
`category_counts`

3. Aggregate all customer total films watched
`total_counts`

4. Identify the top 2 categories for each customer
`top_categories`

5. Calculate each category’s aggregated average rental count
`average_category_count`

6. Calculate the percentile metric for each customer’s top category film count
`top_category_percentile`

7. Generate our first top category insights table using all previously generated tables
`top_category_insights`

8. Generate the 2nd category insights
`second_category_insights`

### 1.2. Category Recommendations

1. Generate a summarised film count table with the category included, I will use this table to rank the films by popularity
`film_counts`

2. Create a previously watched films for the top 2 categories to exclude for each customer
`category_film_exclusions`

3. Finally perform an anti join from the relevant category films on the exclusions and use window functions to keep the top 3 from each category by popularity - be sure to split out the recommendations by category ranking
`category_recommendations`

### 1.3 Actor Insights

1. Create a new base dataset which has a focus on the actor instead of category
`actor_joint_table`

2. Identify the top actor and their respective rental count for each customer based off the ranked rental counts
`top_actor_counts`


### 1.4. Actor Recommendations

1. Generate total actor rental counts to use for film popularity ranking in later steps
`actor_film_counts`

2. Create an updated film exclusions table which includes the previously watched films like I had for the category recommendations - but this time I need to also add in the films which were previously recommended
`actor_film_exclusions`

3. Apply the same ANTI JOIN technique and use a window function to identify the 3 valid film recommendations for customers
`actor_recommendations`

## 2. Category Insights

### 2.1. Create Base Dataset

In [3]:
%%sql
DROP TABLE IF EXISTS complete_joint_dataset;
CREATE TEMP TABLE complete_joint_dataset AS
SELECT
  rental.customer_id,
  inventory.film_id,
  film.title,
  category.name AS category_name,
  -- also included rental_date for sorting purposes
  rental.rental_date
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
  ON rental.inventory_id = inventory.inventory_id
INNER JOIN dvd_rentals.film
  ON inventory.film_id = film.film_id
INNER JOIN dvd_rentals.film_category
  ON film.film_id = film_category.film_id
INNER JOIN dvd_rentals.category
  ON film_category.category_id = category.category_id;

SELECT * FROM complete_joint_dataset limit 10;

 * postgresql://postgres@localhost:5432/
Done.
16044 rows affected.
10 rows affected.


customer_id,film_id,title,category_name,rental_date
130,80,BLANKET BEVERLY,Family,2005-05-24 22:53:30
459,333,FREAKY POCUS,Music,2005-05-24 22:54:33
408,373,GRADUATE LORD,Children,2005-05-24 23:03:39
333,535,LOVE SUICIDES,Horror,2005-05-24 23:04:41
222,450,IDOLS SNATCHERS,Children,2005-05-24 23:05:21
549,613,MYSTIC TRUMAN,Comedy,2005-05-24 23:08:07
269,870,SWARM GOLD,Horror,2005-05-24 23:11:53
239,510,LAWLESS VISION,Animation,2005-05-24 23:31:46
126,565,MATRIX SNOWMAN,Foreign,2005-05-25 00:00:40
399,396,HANGING DEEP,Drama,2005-05-25 00:02:21


### 2.2. Category Counts

I then created a follow-up table which uses the `complete_joint_dataset` to aggregate data and generate a `rental_count` and the `latest rental_date` for ranking purposes downstream.


In [21]:
%%sql
DROP TABLE IF EXISTS category_counts;
CREATE TEMP TABLE category_counts AS
SELECT
  customer_id,
  category_name,
  COUNT(*) AS rental_count,
  MAX(rental_date) AS latest_rental_date
FROM complete_joint_dataset
GROUP BY
  customer_id,
  category_name;

SELECT *
FROM category_counts
WHERE customer_id = 1
ORDER BY
    rental_count DESC,
    latest_rental_date DESC;

 * postgresql://postgres@localhost:5432/
Done.
7741 rows affected.
14 rows affected.


customer_id,category_name,rental_count,latest_rental_date
1,Classics,6,2005-08-19 09:55:16
1,Comedy,5,2005-08-22 19:41:37
1,Drama,4,2005-08-18 03:57:29
1,Animation,2,2005-08-22 20:03:46
1,Sci-Fi,2,2005-08-21 23:33:57
1,New,2,2005-08-19 13:56:54
1,Action,2,2005-08-17 12:37:54
1,Music,2,2005-07-09 16:38:01
1,Sports,2,2005-07-08 07:33:56
1,Family,1,2005-08-02 18:01:38


### 2.3. Total counts

I will then use this `category_counts` table to generate `total_counts` table.


In [22]:
%%sql
DROP TABLE IF EXISTS total_counts;
CREATE TEMP TABLE total_counts AS
SELECT
  customer_id,
  SUM(rental_count) AS total_count
FROM category_counts
GROUP BY
  customer_id;

SELECT *
FROM total_counts
LIMIT 5;

 * postgresql://postgres@localhost:5432/
Done.
599 rows affected.
5 rows affected.


customer_id,total_count
184,23
87,30
477,22
273,35
550,32


### 2.4. Top Categories

I will also split arbitrary ties by preferencing the category which had the most recent `latest_rental_date` value I generated in the `category_counts` for this exact purpose. To further prevent any ties - I will also sort the `category_name` in alphabetical (ascending) order just in case.

In [23]:
%%sql
DROP TABLE IF EXISTS top_categories;
CREATE TEMP TABLE top_categories AS
WITH ranked_cte AS (
  SELECT
    customer_id,
    category_name,
    rental_count,
    DENSE_RANK() OVER (
      PARTITION BY customer_id
      ORDER BY
        rental_count DESC,
        latest_rental_date DESC,
        category_name
    ) AS category_rank
  FROM category_counts
)
SELECT * FROM ranked_cte
WHERE category_rank <= 2;

SELECT *
FROM top_categories
LIMIT 5;

 * postgresql://postgres@localhost:5432/
Done.
1198 rows affected.
5 rows affected.


customer_id,category_name,rental_count,category_rank
1,Classics,6,1
1,Comedy,5,2
2,Sports,5,1
2,Classics,4,2
3,Action,4,1


### 2.5. Average Category Count

Next I will need to use the `category_counts` table to generate the average aggregated rental count for each category rounded down to the nearest integer using the FLOOR function.

In [24]:
%%sql
DROP TABLE IF EXISTS average_category_count;
CREATE TEMP TABLE average_category_count AS
SELECT
  category_name,
  FLOOR(AVG(rental_count)) AS category_average
FROM category_counts
GROUP BY category_name;

SELECT *
FROM average_category_count
ORDER BY
  category_average DESC,
  category_name;

 * postgresql://postgres@localhost:5432/
Done.
16 rows affected.
16 rows affected.


category_name,category_average
Action,2
Animation,2
Classics,2
Documentary,2
Drama,2
Family,2
Foreign,2
Games,2
New,2
Sci-Fi,2


### 2.6. Top category percentile

Now I need to compare each customer’s top category `rental_count` to all other DVD Rental Co customers - I do this using a combination of a LEFT JOIN and a PERCENT_RANK window function ordered by descending rental count to show the required top N% customer insight.

In [26]:
%%sql
DROP TABLE IF EXISTS top_category_percentile;
CREATE TEMP TABLE top_category_percentile AS
WITH calculated_cte AS (
SELECT
  top_categories.customer_id,
  top_categories.category_name AS top_category_name,
  top_categories.rental_count,
  category_counts.category_name,
  top_categories.category_rank,
  PERCENT_RANK() OVER (
    PARTITION BY category_counts.category_name
    ORDER BY category_counts.rental_count DESC
  ) AS raw_percentile_value
FROM category_counts
LEFT JOIN top_categories
  ON category_counts.customer_id = top_categories.customer_id
)
SELECT
  customer_id,
  category_name,
  rental_count,
  category_rank,
  CASE
    WHEN ROUND(100 * raw_percentile_value) = 0 THEN 1
    ELSE ROUND(100 * raw_percentile_value)
  END AS percentile
FROM calculated_cte
WHERE
  category_rank = 1
  AND top_category_name = category_name;

SELECT *
FROM top_category_percentile
LIMIT 10;

 * postgresql://postgres@localhost:5432/
Done.
599 rows affected.
10 rows affected.


customer_id,category_name,rental_count,category_rank,percentile
323,Action,7,1,1.0
506,Action,7,1,1.0
151,Action,6,1,1.0
410,Action,6,1,1.0
126,Action,6,1,1.0
51,Action,6,1,1.0
487,Action,6,1,1.0
363,Action,6,1,1.0
209,Action,6,1,1.0
560,Action,6,1,1.0


### 2.7. 1st Category Insights

Now I'll compile all of previous temporary tables into a single `category_insights` table with what I have so far - I will use most recently generated `top_category_percentile` table as the base and LEFT JOIN average table to generate an `average_comparison` column.

In [27]:
%%sql
DROP TABLE IF EXISTS first_category_insights;
CREATE TEMP TABLE first_category_insights AS
SELECT
  base.customer_id,
  base.category_name,
  base.rental_count,
  base.rental_count - average.category_average AS average_comparison,
  base.percentile
FROM top_category_percentile AS base
LEFT JOIN average_category_count AS average
  ON base.category_name = average.category_name;

SELECT *
FROM first_category_insights
LIMIT 10;

 * postgresql://postgres@localhost:5432/
Done.
599 rows affected.
10 rows affected.


customer_id,category_name,rental_count,average_comparison,percentile
323,Action,7,5,1.0
506,Action,7,5,1.0
151,Action,6,4,1.0
410,Action,6,4,1.0
126,Action,6,4,1.0
51,Action,6,4,1.0
487,Action,6,4,1.0
363,Action,6,4,1.0
209,Action,6,4,1.0
560,Action,6,4,1.0


### 2.8. 2nd category insights

My second ranked category insight is pretty simple as I only need `top_categories` table and the `total_counts `table to process insights.

The only thing to note here is that I’ll need to cast one of fraction components of `total_percentage` column to avoid the dreaded integer floor division.

In [29]:
%%sql
DROP TABLE IF EXISTS second_category_insights;
CREATE TEMP TABLE second_category_insights AS
SELECT
  top_categories.customer_id,
  top_categories.category_name,
  top_categories.rental_count,
  -- need to cast as NUMERIC to avoid INTEGER floor division!
  ROUND(
    100 * top_categories.rental_count::NUMERIC / total_counts.total_count
  ) AS total_percentage
FROM top_categories
LEFT JOIN total_counts
  ON top_categories.customer_id = total_counts.customer_id
WHERE category_rank = 2;

SELECT *
FROM second_category_insights
LIMIT 10;

 * postgresql://postgres@localhost:5432/
Done.
599 rows affected.
10 rows affected.


customer_id,category_name,rental_count,total_percentage
184,Drama,3,13
87,Sci-Fi,3,10
477,Travel,3,14
273,New,4,11
550,Drama,4,13
51,Drama,4,12
394,Documentary,3,14
272,Documentary,3,15
70,Music,2,11
190,Classics,3,11


## 3. Category Recommendations

### 3.1. Film Counts

I wil first generate another total rental count aggregation from base table `complete_joint_dataset` - however this time I will use the `film_id` and title instead of the category - I still need to keep the `category_name` in aggregation - so I will need to use a window function instead of a group by to perform this step.

In [30]:
%%sql
DROP TABLE IF EXISTS film_counts;
CREATE TEMP TABLE film_counts AS
SELECT DISTINCT
  film_id,
  title,
  category_name,
  COUNT(*) OVER (
    PARTITION BY film_id
  ) AS rental_count
FROM complete_joint_dataset;

SELECT *
FROM film_counts
ORDER BY rental_count DESC
LIMIT 10;

 * postgresql://postgres@localhost:5432/
Done.
958 rows affected.
10 rows affected.


film_id,title,category_name,rental_count
103,BUCKET BROTHERHOOD,Travel,34
738,ROCKETEER MOTHER,Foreign,33
331,FORWARD TEMPLE,Games,32
489,JUGGLER HARDLY,Animation,32
767,SCALAWAG DUCK,Music,32
382,GRIT CLOCKWORK,Games,32
730,RIDGEMONT SUBMARINE,New,32
973,WIFE TURN,Documentary,31
621,NETWORK PEAK,Family,31
1000,ZORRO ARK,Comedy,31


### 3.2. Category Film Exclusions

For the next step in recommendation analysis - I will need to generate a table with all of customer’s previously watched films so I don’t recommend them something which they’ve already seen before.

I will use the `complete_joint_dataset` base table to get this information.

In [31]:
%%sql
DROP TABLE IF EXISTS category_film_exclusions;
CREATE TEMP TABLE category_film_exclusions AS
SELECT DISTINCT
  customer_id,
  film_id
FROM complete_joint_dataset;

SELECT *
FROM category_film_exclusions
LIMIT 10;

 * postgresql://postgres@localhost:5432/
Done.
15828 rows affected.
10 rows affected.


customer_id,film_id
596,103
176,121
459,724
375,641
153,730
1,480
291,285
144,93
158,786
211,962


### 3.3. Final category recommendations

To avoid random ties - I will sort by the title alphabetically in case the `rental_count` values are equal in the ORDER BY clause for window function.

I also need to keep `category_rank` column in final output so I can easily identify recommendations for each customer’s preferred categories.

In [32]:
%%sql
DROP TABLE IF EXISTS category_recommendations;
CREATE TEMP TABLE category_recommendations AS
WITH ranked_films_cte AS (
  SELECT
    top_categories.customer_id,
    top_categories.category_name,
    top_categories.category_rank,
    film_counts.film_id,
    film_counts.title,
    film_counts.rental_count,
    DENSE_RANK() OVER (
      PARTITION BY
        top_categories.customer_id,
        top_categories.category_rank
      ORDER BY
        film_counts.rental_count DESC,
        film_counts.title
    ) AS reco_rank
  FROM top_categories
  INNER JOIN film_counts
    ON top_categories.category_name = film_counts.category_name
  WHERE NOT EXISTS (
    SELECT 1
    FROM category_film_exclusions
    WHERE
      category_film_exclusions.customer_id = top_categories.customer_id AND
      category_film_exclusions.film_id = film_counts.film_id
  )
)
SELECT * FROM ranked_films_cte
WHERE reco_rank <= 3;

SELECT *
FROM category_recommendations
WHERE customer_id = 1
ORDER BY category_rank, reco_rank;

 * postgresql://postgres@localhost:5432/
Done.
3594 rows affected.
6 rows affected.


customer_id,category_name,category_rank,film_id,title,rental_count,reco_rank
1,Classics,1,891,TIMBERLAND SKY,31,1
1,Classics,1,358,GILMORE BOILED,28,2
1,Classics,1,951,VOYAGE LEGALLY,28,3
1,Comedy,2,1000,ZORRO ARK,31,1
1,Comedy,2,127,CAT CONEHEADS,30,2
1,Comedy,2,638,OPERATION OPERATION,27,3


## 4. Actor insights

### 4.1. Actor joint table

For this entire analysis on actors - I will need to create a new base table as I will need to introduce the `dvd_rentals.film_actor` and `dvd_rentals.actor` tables to extract all the required data points I need for the final output.

I should also check that the combination of rows in final table is expected because I should see many more rows than previously used in categories insights as there is a many-to-many relationship between `film_id` and `actor_id` as I alluded to earlier in data exploration section of this case study.

I also included the `rental_date` column in this table so I can use it in case there are any ties - just like previous analysis for the top categories piece.

In [33]:
%%sql
-- Actor Insights and Recommendations
DROP TABLE IF EXISTS actor_joint_dataset;
CREATE TEMP TABLE actor_joint_dataset AS
SELECT
  rental.customer_id,
  rental.rental_id,
  rental.rental_date,
  film.film_id,
  film.title,
  actor.actor_id,
  actor.first_name,
  actor.last_name
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
  ON rental.inventory_id = inventory.inventory_id
INNER JOIN dvd_rentals.film
  ON inventory.film_id = film.film_id
-- different to our previous base table as we know use actor tables
INNER JOIN dvd_rentals.film_actor
  ON film.film_id = film_actor.film_id
INNER JOIN dvd_rentals.actor
  ON film_actor.actor_id = actor.actor_id;

-- show the counts and count distinct of a few important columns
SELECT
  COUNT(*) AS total_row_count,
  COUNT(DISTINCT rental_id) AS unique_rental_id,
  COUNT(DISTINCT film_id) AS unique_film_id,
  COUNT(DISTINCT actor_id) AS unique_actor_id,
  COUNT(DISTINCT customer_id) AS unique_customer_id
FROM actor_joint_dataset;

 * postgresql://postgres@localhost:5432/
Done.
87980 rows affected.
1 rows affected.


total_row_count,unique_rental_id,unique_film_id,unique_actor_id,unique_customer_id
87980,16004,955,200,599


In [34]:
%%sql
-- show the first 10 rows
SELECT *
FROM actor_joint_dataset
LIMIT 10;

 * postgresql://postgres@localhost:5432/
10 rows affected.


customer_id,rental_id,rental_date,film_id,title,actor_id,first_name,last_name
130,1,2005-05-24 22:53:30,80,BLANKET BEVERLY,200,THORA,TEMPLE
130,1,2005-05-24 22:53:30,80,BLANKET BEVERLY,193,BURT,TEMPLE
130,1,2005-05-24 22:53:30,80,BLANKET BEVERLY,173,ALAN,DREYFUSS
130,1,2005-05-24 22:53:30,80,BLANKET BEVERLY,16,FRED,COSTNER
459,2,2005-05-24 22:54:33,333,FREAKY POCUS,147,FAY,WINSLET
459,2,2005-05-24 22:54:33,333,FREAKY POCUS,127,KEVIN,GARLAND
459,2,2005-05-24 22:54:33,333,FREAKY POCUS,105,SIDNEY,CROWE
459,2,2005-05-24 22:54:33,333,FREAKY POCUS,103,MATTHEW,LEIGH
459,2,2005-05-24 22:54:33,333,FREAKY POCUS,42,TOM,MIRANDA
408,3,2005-05-24 23:03:39,373,GRADUATE LORD,140,WHOOPI,HURT


### 4.2. Top actor counts

I can now generate rental counts per actor and since I are only interested in the top actor for each of customers - I can also perform a filter step to just keep the top actor records and counts for downstream insights.

In [35]:
%%sql
DROP TABLE IF EXISTS top_actor_counts;
CREATE TEMP TABLE top_actor_counts AS
WITH actor_counts AS (
  SELECT
    customer_id,
    actor_id,
    first_name,
    last_name,
    COUNT(*) AS rental_count,
    -- I also generate the latest_rental_date just like category insight
    MAX(rental_date) AS latest_rental_date
  FROM actor_joint_dataset
  GROUP BY
    customer_id,
    actor_id,
    first_name,
    last_name
),
ranked_actor_counts AS (
  SELECT
    actor_counts.*,
    DENSE_RANK() OVER (
      PARTITION BY customer_id
      ORDER BY
        rental_count DESC,
        latest_rental_date DESC,
        -- just in case I have any further ties, I'll throw in the names.
        first_name,
        last_name
    ) AS actor_rank
  FROM actor_counts
)
SELECT
  customer_id,
  actor_id,
  first_name,
  last_name,
  rental_count
FROM ranked_actor_counts
WHERE actor_rank = 1;

SELECT *
FROM top_actor_counts
LIMIT 10;

 * postgresql://postgres@localhost:5432/
Done.
599 rows affected.
10 rows affected.


customer_id,actor_id,first_name,last_name,rental_count
1,37,VAL,BOLGER,6
2,107,GINA,DEGENERES,5
3,150,JAYNE,NOLTE,4
4,102,WALTER,TORN,4
5,12,KARL,BERRY,4
6,191,GREGORY,GOODING,4
7,65,ANGELA,HUDSON,5
8,167,LAURENCE,BULLOCK,5
9,23,SANDRA,KILMER,3
10,12,KARL,BERRY,4


## 5. Actor recommendations

### 5.1. Actor film counts

I need to generate aggregated total rental counts across all customers by `actor_id` and `film_id` so I can join onto `top_actor_counts` table.

In [36]:
%%sql
DROP TABLE IF EXISTS actor_film_counts;
CREATE TEMP TABLE actor_film_counts AS
WITH film_counts AS (
  SELECT
    film_id,
    COUNT(DISTINCT rental_id) AS rental_count
  FROM actor_joint_dataset
  GROUP BY film_id
)
SELECT DISTINCT
  actor_joint_dataset.film_id,
  actor_joint_dataset.actor_id,
  actor_joint_dataset.title,
  film_counts.rental_count
FROM actor_joint_dataset
LEFT JOIN film_counts
  ON actor_joint_dataset.film_id = film_counts.film_id;

SELECT *
FROM actor_film_counts
LIMIT 10;

 * postgresql://postgres@localhost:5432/
Done.
5246 rows affected.
10 rows affected.


film_id,actor_id,title,rental_count
930,178,VACATION BOONDOCK,21
347,198,GAMES BOWFINGER,14
190,91,CREEPERS KANE,14
812,85,SMOKING BARBARELLA,18
485,2,JERSEY SASSY,6
683,177,PITY BOUND,26
714,115,RANDOM GO,9
226,29,DESTINY SATURDAY,16
125,144,CASSIDY WYOMING,6
533,189,LOST BIRD,12


### 5.2. Actor film exclusions

I will perform the same steps I used to create the `category_film_exclusions` table - however I need to UNION exclusions with the relevant category recommendations that I have already given customers.

The rationale behind this - customers would not want to receive a recommendation for the same film twice in the same email.

In [37]:
%%sql
DROP TABLE IF EXISTS actor_film_exclusions;
CREATE TEMP TABLE actor_film_exclusions AS
-- repeat the first steps as per the category exclusions
-- I'll use our original complete_joint_dataset as the base here
(
  SELECT DISTINCT
    customer_id,
    film_id
  FROM complete_joint_dataset
)
-- I use a UNION to combine the previously watched and the recommended films.
UNION
(
  SELECT DISTINCT
    customer_id,
    film_id
  FROM category_recommendations
);

SELECT *
FROM actor_film_exclusions
LIMIT 10;

 * postgresql://postgres@localhost:5432/
Done.
19422 rows affected.
10 rows affected.


customer_id,film_id
493,567
114,789
596,103
176,121
459,724
375,641
153,730
291,285
1,480
144,93


### 5.3. Final actor recommendations

This time I will use `top_actor_counts`, `actor_film_counts` and `actor_film_exclusions` tables for analysis.

In [38]:
%%sql
DROP TABLE IF EXISTS actor_recommendations;
CREATE TEMP TABLE actor_recommendations AS
WITH ranked_actor_films_cte AS (
  SELECT
    top_actor_counts.customer_id,
    top_actor_counts.first_name,
    top_actor_counts.last_name,
    top_actor_counts.rental_count,
    actor_film_counts.title,
    actor_film_counts.film_id,
    actor_film_counts.actor_id,
    DENSE_RANK() OVER (
      PARTITION BY
        top_actor_counts.customer_id
      ORDER BY
        actor_film_counts.rental_count DESC,
        actor_film_counts.title
    ) AS reco_rank
  FROM top_actor_counts
  INNER JOIN actor_film_counts
    -- join on actor_id instead of category_name!
    ON top_actor_counts.actor_id = actor_film_counts.actor_id
  -- This is a tricky anti-join where we need to "join" on 2 different tables!
  WHERE NOT EXISTS (
    SELECT 1
    FROM actor_film_exclusions
    WHERE
      actor_film_exclusions.customer_id = top_actor_counts.customer_id AND
      actor_film_exclusions.film_id = actor_film_counts.film_id
  )
)
SELECT * FROM ranked_actor_films_cte
WHERE reco_rank <= 3;

SELECT *
FROM actor_recommendations
ORDER BY customer_id, reco_rank
LIMIT 15;

 * postgresql://postgres@localhost:5432/
Done.
1797 rows affected.
15 rows affected.


customer_id,first_name,last_name,rental_count,title,film_id,actor_id,reco_rank
1,VAL,BOLGER,6,PRIMARY GLASS,697,37,1
1,VAL,BOLGER,6,ALASKA PHANTOM,12,37,2
1,VAL,BOLGER,6,METROPOLIS COMA,572,37,3
2,GINA,DEGENERES,5,GOODFELLAS SALUTE,369,107,1
2,GINA,DEGENERES,5,WIFE TURN,973,107,2
2,GINA,DEGENERES,5,DOGMA FAMILY,239,107,3
3,JAYNE,NOLTE,4,SWEETHEARTS SUSPECTS,873,150,1
3,JAYNE,NOLTE,4,DANCING FEVER,206,150,2
3,JAYNE,NOLTE,4,INVASION CYCLONE,468,150,3
4,WALTER,TORN,4,CURTAIN VIDEOTAPE,200,102,1


# Report

## 6. Key table outputs

### 6.1. Customer level insights

* `first_category_insights`

In [39]:
%%sql
SELECT *
FROM first_category_insights
LIMIT 10;

 * postgresql://postgres@localhost:5432/
10 rows affected.


customer_id,category_name,rental_count,average_comparison,percentile
323,Action,7,5,1.0
506,Action,7,5,1.0
151,Action,6,4,1.0
410,Action,6,4,1.0
126,Action,6,4,1.0
51,Action,6,4,1.0
487,Action,6,4,1.0
363,Action,6,4,1.0
209,Action,6,4,1.0
560,Action,6,4,1.0


* `second_category_insights`

In [40]:
%%sql
SELECT *
FROM second_category_insights
LIMIT 10;

 * postgresql://postgres@localhost:5432/
10 rows affected.


customer_id,category_name,rental_count,total_percentage
184,Drama,3,13
87,Sci-Fi,3,10
477,Travel,3,14
273,New,4,11
550,Drama,4,13
51,Drama,4,12
394,Documentary,3,14
272,Documentary,3,15
70,Music,2,11
190,Classics,3,11


* `top_actor_counts`

In [41]:
%%sql
SELECT *
FROM top_actor_counts
LIMIT 10;

 * postgresql://postgres@localhost:5432/
10 rows affected.


customer_id,actor_id,first_name,last_name,rental_count
1,37,VAL,BOLGER,6
2,107,GINA,DEGENERES,5
3,150,JAYNE,NOLTE,4
4,102,WALTER,TORN,4
5,12,KARL,BERRY,4
6,191,GREGORY,GOODING,4
7,65,ANGELA,HUDSON,5
8,167,LAURENCE,BULLOCK,5
9,23,SANDRA,KILMER,3
10,12,KARL,BERRY,4


### 6.2. Recommendations

* `category_recommendations`

In [42]:
%%sql
SELECT *
FROM category_recommendations
WHERE customer_id = 1
ORDER BY category_rank, reco_rank;

 * postgresql://postgres@localhost:5432/
6 rows affected.


customer_id,category_name,category_rank,film_id,title,rental_count,reco_rank
1,Classics,1,891,TIMBERLAND SKY,31,1
1,Classics,1,358,GILMORE BOILED,28,2
1,Classics,1,951,VOYAGE LEGALLY,28,3
1,Comedy,2,1000,ZORRO ARK,31,1
1,Comedy,2,127,CAT CONEHEADS,30,2
1,Comedy,2,638,OPERATION OPERATION,27,3


* `actor_recommendations`

In [43]:
%%sql
SELECT *
FROM actor_recommendations
ORDER BY customer_id, reco_rank
LIMIT 15;

 * postgresql://postgres@localhost:5432/
15 rows affected.


customer_id,first_name,last_name,rental_count,title,film_id,actor_id,reco_rank
1,VAL,BOLGER,6,PRIMARY GLASS,697,37,1
1,VAL,BOLGER,6,ALASKA PHANTOM,12,37,2
1,VAL,BOLGER,6,METROPOLIS COMA,572,37,3
2,GINA,DEGENERES,5,GOODFELLAS SALUTE,369,107,1
2,GINA,DEGENERES,5,WIFE TURN,973,107,2
2,GINA,DEGENERES,5,DOGMA FAMILY,239,107,3
3,JAYNE,NOLTE,4,SWEETHEARTS SUSPECTS,873,150,1
3,JAYNE,NOLTE,4,DANCING FEVER,206,150,2
3,JAYNE,NOLTE,4,INVASION CYCLONE,468,150,3
4,WALTER,TORN,4,CURTAIN VIDEOTAPE,200,102,1


### 6.3. Final transformations

In [44]:
%%sql
DROP TABLE IF EXISTS final_data_asset;
CREATE TEMP TABLE final_data_asset AS
WITH first_category AS (
  SELECT
    customer_id,
    category_name,
    CONCAT(
      'You''ve watched ', rental_count, ' ', category_name,
      ' films, that''s ', average_comparison,
      ' more than the DVD Rental Co average and puts you in the top ',
      percentile, '% of ', category_name, ' gurus!'
    ) AS insight
  FROM first_category_insights
),
second_category AS (
  SELECT
    customer_id,
    category_name,
    CONCAT(
      'You''ve watched ', rental_count, ' ', category_name,
      ' films making up ', total_percentage,
      '% of your entire viewing history!'
    ) AS insight
  FROM second_category_insights
),
top_actor AS (
  SELECT
    customer_id,
    -- use INITCAP to transform names into Title case
    CONCAT(INITCAP(first_name), ' ', INITCAP(last_name)) AS actor_name,
    CONCAT(
      'You''ve watched ', rental_count, ' films featuring ',
      INITCAP(first_name), ' ', INITCAP(last_name),
      '! Here are some other films ', INITCAP(first_name),
      ' stars in that might interest you!'
    ) AS insight
  FROM top_actor_counts
),
adjusted_title_case_category_recommendations AS (
  SELECT
    customer_id,
    INITCAP(title) AS title,
    category_rank,
    reco_rank
  FROM category_recommendations
),
wide_category_recommendations AS (
  SELECT
    customer_id,
    MAX(CASE WHEN category_rank = 1  AND reco_rank = 1
      THEN title END) AS cat_1_reco_1,
    MAX(CASE WHEN category_rank = 1  AND reco_rank = 2
      THEN title END) AS cat_1_reco_2,
    MAX(CASE WHEN category_rank = 1  AND reco_rank = 3
      THEN title END) AS cat_1_reco_3,
    MAX(CASE WHEN category_rank = 2  AND reco_rank = 1
      THEN title END) AS cat_2_reco_1,
    MAX(CASE WHEN category_rank = 2  AND reco_rank = 2
      THEN title END) AS cat_2_reco_2,
    MAX(CASE WHEN category_rank = 2  AND reco_rank = 3
      THEN title END) AS cat_2_reco_3
  FROM adjusted_title_case_category_recommendations
  GROUP BY customer_id
),
adjusted_title_case_actor_recommendations AS (
  SELECT
    customer_id,
    INITCAP(title) AS title,
    reco_rank
  FROM actor_recommendations
),
wide_actor_recommendations AS (
  SELECT
    customer_id,
    MAX(CASE WHEN reco_rank = 1 THEN title END) AS actor_reco_1,
    MAX(CASE WHEN reco_rank = 2 THEN title END) AS actor_reco_2,
    MAX(CASE WHEN reco_rank = 3 THEN title END) AS actor_reco_3
  FROM adjusted_title_case_actor_recommendations
  GROUP BY customer_id
),
final_output AS (
  SELECT
    t1.customer_id,
    t1.category_name AS cat_1,
    t4.cat_1_reco_1,
    t4.cat_1_reco_2,
    t4.cat_1_reco_3,
    t2.category_name AS cat_2,
    t4.cat_2_reco_1,
    t4.cat_2_reco_2,
    t4.cat_2_reco_3,
    t3.actor_name AS actor,
    t5.actor_reco_1,
    t5.actor_reco_2,
    t5.actor_reco_3,
    t1.insight AS insight_cat_1,
    t2.insight AS insight_cat_2,
    t3.insight AS insight_actor
FROM first_category AS t1
INNER JOIN second_category AS t2
  ON t1.customer_id = t2.customer_id
INNER JOIN top_actor t3
  ON t1.customer_id = t3.customer_id
INNER JOIN wide_category_recommendations AS t4
  ON t1.customer_id = t4.customer_id
INNER JOIN wide_actor_recommendations AS t5
  ON t1.customer_id = t5.customer_id
)
SELECT * FROM final_output;

SELECT * FROM final_data_asset
LIMIT 5;

 * postgresql://postgres@localhost:5432/
Done.
599 rows affected.
5 rows affected.


customer_id,cat_1,cat_1_reco_1,cat_1_reco_2,cat_1_reco_3,cat_2,cat_2_reco_1,cat_2_reco_2,cat_2_reco_3,actor,actor_reco_1,actor_reco_2,actor_reco_3,insight_cat_1,insight_cat_2,insight_actor
1,Classics,Timberland Sky,Gilmore Boiled,Voyage Legally,Comedy,Zorro Ark,Cat Coneheads,Operation Operation,Val Bolger,Primary Glass,Alaska Phantom,Metropolis Coma,"You've watched 6 Classics films, that's 4 more than the DVD Rental Co average and puts you in the top 1% of Classics gurus!",You've watched 5 Comedy films making up 16% of your entire viewing history!,You've watched 6 films featuring Val Bolger! Here are some other films Val stars in that might interest you!
2,Sports,Gleaming Jawbreaker,Talented Homicide,Roses Treasure,Classics,Frost Head,Gilmore Boiled,Voyage Legally,Gina Degeneres,Goodfellas Salute,Wife Turn,Dogma Family,"You've watched 5 Sports films, that's 3 more than the DVD Rental Co average and puts you in the top 2% of Sports gurus!",You've watched 4 Classics films making up 15% of your entire viewing history!,You've watched 5 films featuring Gina Degeneres! Here are some other films Gina stars in that might interest you!
3,Action,Rugrats Shakespeare,Suspects Quills,Handicap Boondock,Sci-Fi,Goodfellas Salute,English Bulworth,Graffiti Love,Jayne Nolte,Sweethearts Suspects,Dancing Fever,Invasion Cyclone,"You've watched 4 Action films, that's 2 more than the DVD Rental Co average and puts you in the top 4% of Action gurus!",You've watched 3 Sci-Fi films making up 12% of your entire viewing history!,You've watched 4 films featuring Jayne Nolte! Here are some other films Jayne stars in that might interest you!
4,Horror,Pulp Beverly,Family Sweet,Swarm Gold,Drama,Hobbit Alien,Harry Idaho,Witches Panic,Walter Torn,Curtain Videotape,Lies Treatment,Nightmare Chill,"You've watched 3 Horror films, that's 2 more than the DVD Rental Co average and puts you in the top 8% of Horror gurus!",You've watched 2 Drama films making up 9% of your entire viewing history!,You've watched 4 films featuring Walter Torn! Here are some other films Walter stars in that might interest you!
5,Classics,Timberland Sky,Frost Head,Gilmore Boiled,Animation,Juggler Hardly,Dogma Family,Storm Happiness,Karl Berry,Virginian Pluto,Stagecoach Armageddon,Telemark Heartbreakers,"You've watched 7 Classics films, that's 5 more than the DVD Rental Co average and puts you in the top 1% of Classics gurus!",You've watched 6 Animation films making up 16% of your entire viewing history!,You've watched 4 films featuring Karl Berry! Here are some other films Karl stars in that might interest you!


**Email output for `customer_id` = 5**

<img src="images/email-output.png" width=40% height=40%>